# Word2Vec - Dirac notation

<font color='red'>STATUS - VERY PRELIMINARY DRAFT</font>

## Introduction

Here I am trying to build my understanding of Language Modeling based on Word2Vec algorithm.<br>
I learn from  
* Yoav Goldberg's book "Neural Network Methods for Natural Language Processing",
* [Stanford NLP course (cs224n)](http://web.stanford.edu/class/cs224n/) ([Lecture 2](https://www.youtube.com/watch?v=ERibwqs9p38))
* Original Papers by Mikolov et al. [Efficient Estimation of Word Representations in Vector Space](https://arxiv.org/abs/1301.3781), and [Distributed Representations of Words and Phrases and their Compositionality](https://arxiv.org/abs/1310.4546)
* Implementation Source code (C, Linux) from ([google archive](https://code.google.com/archive/p/word2vec/))
* [fast.ai](http://www.fast.ai/) library
* [McCormick, C. (2016, April 19). Word2Vec Tutorial - The Skip-Gram Model](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/)
* [McCormick, C. (2017, January 11). Word2Vec Tutorial Part 2 - Negative Sampling](http://mccormickml.com/2017/01/11/word2vec-tutorial-part-2-negative-sampling/)
* [Word2Vec Resources collection by McCormick](http://mccormickml.com/2016/04/27/word2vec-resources/#implementations); McCormick extensively annotated the google's [word2vec package](https://github.com/chrisjmccormick/word2vec_commented/), which is a huge help 
* "Sequence Models" as a part of "Deep Learning Specialization" in Coursera

## Problem Statement - The Language Modeling Task

Language modeling is the task of assigning probabilities to word sequences (sentences) in a language, or assigning probability of a given word (or sequence of words) to follow a sequence of words.<br>

NLP tasks (translation, transcription, etc.) handling systems usually produce several hypotheses (several word sequences). These hypotheses are scored according to their probabilities (language model), and the hypotheses having the highest score is selected. <br>

Language modeling is also frequently used for unsupervised pre-training. In cases when we do not have large enough amount of annotated data for our specific task, we use the language modeling, that can be trained on huge amount of unannotated text, to extract useful features that we then use while training for the our specific task.

Probability of a given sequence of words $w_{1:n}$ can be factorized based on probability chain-rule

$$P(w_{1:n})=P(w_{1})P(w_{2}|w_{1})P(w_{3}|w_{1:2})P(w_{4}|w_{1:3})...P(w_{n}|w_{1:n-1})$$

A resonable assumption is to say that the next word in a sequence depends only on the last $k$ words, but not on entire sequence of the previous words (also called $k$th order Markov assumption):
$$P(w_{i+1}|w_{1:i})\approx{P(w_{i+1}|w_{i-k:i})}$$

With this assumption, our chain of probabilities is further simplified:
$$P(w_{1:n})=P(w_{1})P(w_{2}|w_{1})P(w_{3}|w_{1:2})P(w_{4}|w_{1:3})...\prod_{i>k}^n{P(w_{i}|w_{i-k:i-1})}$$

Then, our task is to estimate $P(w_{i}|w_{i-k:i-1})$ given big long text. Practically, to avoid computationally expensive normalization, people use scores based on counts instead of probabilities. 

## Word2Vec

Word2Vec is a software package (in C for linux) that starts from language modeling by a neural network and modifies it to produce faster results. It implements two different context representations (CBOW and Skip-Gram) and two different optimization objectives (Negative-Sampling and Hierarchical Softmax). The explanation will follow below...

Word2Vec package comes with demo-train-big-model-v1.sh script that downloads about 8 billion (8 tera-) words from various public sources (wikipedia, etc.), and normalizes the text by converting to lowercase and removing special characters.  This way we have a big long text as an input for our model. (Mikolov et al. trained their model on 100 billion (100 tera-) words from a Google News dataset.)

The order of words in this text is not random - the words are sequenced in a way that creates a meaning. <br>
If $p(x_{k},x_{k-1},...)$ represents a probability to find words $x_{k},x_{k-1},...$ in a given order, this value should have a maximum for the meaningfull sequence of words. Probability to find a "random" sequence of words in a real text is very small. 


The objective is to maximize (over our big long text) the objective function
$$J^{'}(\theta)=\prod_{t=1}^{T}\prod_{\underset{{j}\neq{o}}{-m<=j<=m}}p(w_{t+j}|w_{t};\theta)$$ 
In other words the probabilities that maximize the $J^{'}(\theta)$ over the long text should represent the correct probabilistic relations between the words in this text


After taking a $log$ and negating the sign, our objective becomes to minimize the negative $log$ likelihood
$$J(\theta)=-\frac{1}{T}\sum_{t=1}^{T}\sum_{\underset{{j}\neq{o}}{-m<=j<=m}}log({p(w_{t+j}|w_{t};\theta))}$$
$\theta$ is a vector representation of the words (embedding vectors); $T$ all possible positions of a central word (so, $\frac{1}{T}$ is a "per word" normalization)

Probability to find a word $o$ as a context (in vicinity) of a center word $c$ is defined as 
$$p(o|c)=\frac{exp(U_{o}^{T}\cdotp{V_{c}})}{\sum_{w=1}^{V}exp(U_{w}^{T}\cdotp{V_{c}})}$$
In other words, we model the probabilities by ["softmax regression classifier"](http://ufldl.stanford.edu/tutorial/supervised/SoftmaxRegression/), where the weights of the regression classifier are the context word vectors. 
> question: may be considering the $p(o|c)$ as a transition probability from $c$ to $o$ can be more productive?

<br>
we take a big long amount of text, we go through each position in the text, for each position of the text we look at a window of size $2m$, we calculate a probability distribution of a word appearing in a context of a central word 
<br>





TBD: check how sparse are the word vectors (how many non-zero coefficients)

Scale estimations: <br>
A word vector is assigned for each word from the vacabulary, so 
* for vacabulary size of 1 million words and vector dimension of 500 the model has to learn these 500 million parameters for central word vectors and 500 million paramters for context vectors, and assuming 4-bytes for each vector's element, we end-up with 4TB (terra bytes) of memory for word vectors..! 
* For 10K words and vector dimension of 100, we have 1 million parameters for central word vectors and 1 million paramters for the context word vectors to learn and 8MB (mega bytes) of memory required for the word vectors.

To get things in proportion: number of paramters in [AlexNet](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf) is 62,378,344 (good explanation is [here](https://www.learnopencv.com/number-of-parameters-and-tensor-sizes-in-convolutional-neural-network/)).<br>
As a reminder: our text's size is tens or hundreds teras of words.

TBD: Here I will discuss the learning strategy selected in word2vec tool and compare it with the "standard" approach of having training and validation sets and iterative, epoch by epoch, process of learning over the entire trainig set, and calculating the achieved accuracy based on the validation set. 

The second paper of Mikolov et al. proposes three heuristics for optimization of the learning process:
1. Treating common word pairs or phrases as single “words” in their model. (we will skip the explanation of finding the common word phrases; this inflates the vocabulary, but improves the quality of the model) 
2. Subsampling frequent words to decrease the number of training examples.
3. Modifying the optimization objective with a technique they called “Negative Sampling”, which causes each training sample to update only a small percentage of the model’s weights.


## Dirac Notation

Representing of a word by a vector and calculating mutual probabilities based on some kind of vectors product is similar to what we do in Quantum Mechanics (QM). I will try to understand how far can we take this similarity.

The first postulate of QM: physical state of a system is represented by a vector Hilbert space<br>
Hilbert space: normalized square integralable functions

Dirac notation - symbols to represent vectors, inner products and operators 

Ket: $|\Psi>$

Bra: $<\Psi|$ - some form of a conjugate to the Ket vector

Braket : $<\Psi|\Phi>$ inner product within Hilbert space

$d$-dimensional vectors for $V$ many words

The idea of Distributed Representations has been introduced to Neural Networks community by [Hinton et al. ](https://dl.acm.org/citation.cfm?id=104287). They proposed to represent each entity by a vector. The meaning of the entity and its relations to other entities are derived from similarities between corresponding the vectors. Compare it with the first postulate of QM!

However, here are the differences: 
* Vectors optimizing the `softmax regression classifier` are defined in real space $\mathbb{R}^d$ and fixed up to common additive constant: value of the softmax does not change if we add (or subtract) the same constant value to (from) all vector parameters. It also looks that we cannot normilize simulteneously all the word vectors in terms of a 'common sense', $L_1$ or $L_2$, norm. `softmax` always (for all vectors in $\mathbb{R}^d$) return positive normalized values that are interpreted by the model as probabilities.

* Vectors in QM are defined in complex space $\mathbb{C}^d$ and normalized in $L_2$. Probabilities are defined by squared, in complex space, braket:  $<\Psi|\Phi><\Phi|\Psi>$

I am planning to read ["Quantum-inspired Complex Word Embedding" by Qiuchi Li, Sagar Uprety, Benyou Wang, Dawei Song](https://arxiv.org/abs/1805.11351)

> Distributional hypothesis says: words that appear in similar contexts have similar meanings. 
> Word2Vec assigned similar vectors to the words that appear in similar contexts.
> Therefore words that have similar word2vec vectors have similar meanings. 